In [1]:
import pyatmos
atmos = pyatmos.Simulation(docker_image="registry.gitlab.com/frontierdevelopmentlab/astrobiology/pyatmos",
                           DEBUG=True)
atmos.start()

Initializing Docker...
Pulling latest image... registry.gitlab.com/frontierdevelopmentlab/astrobiology/pyatmos
Initialization complete: 2018-08-01 14:18:47
Starting Docker container...
Container 'eloquent_ardinghelli' running at 2018-08-01 14:18:48.


In [2]:
base_conc = 0.25
for it in range(0, 1):
    concentration = base_conc + float(it)/100
    print(it, concentration)
    
    args = {
        'species_concentrations' : {'O2' : concentration},
        'max_photochem_iterations' : 10000, 
        'max_clima_steps' : 2, 
        'output_directory' : '/Users/Will/Documents/FDL/results/run_4_{0}'.format(it),
    }
    if it > 0:
        args['previous_photochem_solution'] = '/Users/Will/Documents/FDL/results/run_4_{0}/out.dist'.format(it-1)
        args['previous_clima_solution'] = '/Users/Will/Documents/FDL/results/run_4_{0}/TempOut.dat'.format(it-1)
    print('running atmos with args:')
    print(args)
    atmos.run(**args)
    print('')

0 0.25
running atmos with args:
{'species_concentrations': {'O2': 0.25}, 'max_photochem_iterations': 10000, 'max_clima_steps': 2, 'output_directory': '/Users/Will/Documents/FDL/results/run_4_0'}
DEBUG run: cd /code/atmos
DEBUG _copy_container_file: docker cp eloquent_ardinghelli:/code/atmos/PHOTOCHEM/INPUTFILES/species.dat /var/folders/t1/xhhxt12502bcgyn1r77j10m00000gn/T/tmp_7_wpu10
DEBUG _write_container_file: docker cp /var/folders/t1/xhhxt12502bcgyn1r77j10m00000gn/T/tmphwbm6a3_ eloquent_ardinghelli:/code/atmos/PHOTOCHEM/INPUTFILES/species.dat
Modified species file with:
{'O2': 0.25}
DEBUG _copy_container_file: docker cp eloquent_ardinghelli:/code/atmos/PHOTOCHEM/OUTPUT/out.out /var/folders/t1/xhhxt12502bcgyn1r77j10m00000gn/T/tmp8xnzkl9a
photochem finished after 161 iterations
DEBUG _run_photochem: photochem took 31 seconds
DEBUG _copy_container_file: docker cp eloquent_ardinghelli:/code/atmos/PHOTOCHEM/OUTPUT/out.out /Users/Will/Documents/FDL/results/run_4_0
DEBUG _copy_container_fi

In [3]:
atmos.get_surface_temperature('/Users/Will/Documents/FDL/results/run_3_1/parsed_clima_final.csv')


287.97

In [4]:
atmos.get_surface_pressure('/Users/Will/Documents/FDL/results/run_3_1/parsed_clima_final.csv')

1.0298

In [5]:
import pandas as pd
df = pd.read_csv('/Users/Will/Documents/FDL/results/run_4_0/parsed_clima_iterations.csv')

In [6]:
n_iterations = df['NST'].max()
new_df = df[df['NST'] == n_iterations]

In [14]:
float(new_df['DIVFrms'])

0.00186

In [8]:
new_df.columns

Index(['NST', 'JCONV', 'CHG', 'dt0', 'DIVF(1)', 'DIVFrms', 'DT(ND)', 'T(ND)'], dtype='object')

In [9]:
new_df.keys()

Index(['NST', 'JCONV', 'CHG', 'dt0', 'DIVF(1)', 'DIVFrms', 'DT(ND)', 'T(ND)'], dtype='object')

In [10]:
new_df.JCONV

1    74.0
Name: JCONV, dtype: float64

In [43]:
atmos.close()

Exiting...
Container killed.


In [77]:
import pandas as pd
counter = 0
file_name = '/Users/Will/Documents/FDL/results/run2/species.dat'
data = []
columns = []
with open(file_name, 'r') as file:
    for line in file.readlines():
        line = line.rstrip('\n\r')
        line = ' '.join(line.split())

        # deal with the column headings
        if 'LONG-LIVED' in line:
            
            columns = line.replace('*','')
            columns = 'order species' + columns
            columns = columns.split()
            #print(columns)

        # deal with the rows
        if not (line.startswith('*')):
            if len(line)>0:
                #line = ' '.join(line.split()) # remove extra spaces
                line = str(counter)+ ' ' + line
                counter +=1 # keep track of order (may be important!)
                #print(line)
                
                data_parts = []
                for x in line.split():
                    try:
                        data_parts.append(int(x))
                    except ValueError:
                        try:
                            data_parts.append(float(x))
                        except ValueError:
                            data_parts.append(x)
                    
                        
                
                
                data.append(data_parts)
                
df = pd.DataFrame(data=data, columns=columns)

In [113]:
def speciesfile_to_df(species_filename):
    '''
    Takes the path to a species.dat file and returns a pandas dataframe
    Missing data entries are converted to NaN
    Ints are converted to integer
    Floats are converted to float
    Other entries remain as string
    Args:
        species_filename: string, path to the species.dat filename
    Returns:
        two pandas dataframes containing the relevant information from species.dat
        longlived_df contains the long-lived species
        other_df contains the other species
    '''
    import pandas as pd
    counter = 0
    file_name = '/Users/Will/Documents/FDL/results/run2/species.dat'
    data = []
    columns = []
    with open(file_name, 'r') as file:
        for line in file.readlines():
            line = line.rstrip('\n\r')
            line = ' '.join(line.split())

            # deal with the column headings
            if 'LONG-LIVED' in line:

                columns = line.replace('*','')
                columns = 'order species' + columns
                columns = columns.split()

            # deal with the rows
            if not line.startswith('*'):
                
                if len(line)>0:
                    line = str(counter)+ ' ' + line
                    counter +=1 # keep track of order (may be important!)

                    # convert types to int or float if possible
                    '''
                    data_parts = []
                    for x in line.split():
                        try:
                            data_parts.append(int(x))
                        except ValueError:
                            try:
                                data_parts.append(float(x))
                            except ValueError:
                                data_parts.append(x)
                    data.append(data_parts)
                    '''
                    data.append(line.split())
                
    # Create the dataframe            
    df = pd.DataFrame(data=data, columns=columns)
                
    # set index to be the species type
    df.index = df['species']
    df.drop(columns=['species'], inplace=True)
                
    # split the df into two, one for the long-lived species, and a second for the other rest
    longlived_df = df[df['LONG-LIVED'] == 'LL']
    other_df = df[df['LONG-LIVED'] != 'LL']

    longlived_df = longlived_df.apply(pd.to_numeric, errors='ignore')
    other_df = other_df.apply(pd.to_numeric, errors='ignore')
    
    # remove extraneous columns in other_df
    other_df.drop(columns=['VDEP0', 'FIXEDMR', 'SGFLUX', 'DISTH', 'MBOUND', 'SMFLUX', 'VEFF0'], inplace=True)
   
    other_df.rename(columns={'LBOUND' : 'FIXEDMR'}, inplace=True)
    
    #pd.to_numeric(df, errors='ignore')
    return longlived_df, other_df

In [109]:
longlived_df, other_df = speciesfile_to_df('/Users/Will/Documents/FDL/results/run2/species.dat')

In [110]:
species_to_change = {'O2'  : 0.25 }
fluxes_to_change  = {'CO2' : 6.875E+08}

In [114]:

other_df.head()

,order,LONG-LIVED,O,H,C,S,N,CL,FIXEDMR
species,,,,,,,,,
HNO2,61,SL,2,1,0,0,1,0,NaN
O1D,62,SL,1,0,0,0,0,0,NaN
CH21,63,SL,0,2,1,0,0,0,NaN
CH23,64,SL,0,2,1,0,0,0,NaN
C2H5,65,SL,0,5,2,0,0,0,NaN


In [115]:
#help(species_df.drop)
longlived_df.head()
#species_df.dtypes

,order,LONG-LIVED,O,H,C,S,N,CL,LBOUND,VDEP0,FIXEDMR,SGFLUX,DISTH,MBOUND,SMFLUX,VEFF0
species,,,,,,,,,,,,,,,,
O,0,LL,1,0,0,0,0,0,0,1.0,0.00,0.0,0.0,0,0.0,0.0
O2,1,LL,2,0,0,0,0,0,1,0.0,0.27,0.0,0.0,0,0.0,0.0
H2O,2,LL,1,2,0,0,0,0,0,0.0,0.00,0.0,0.0,0,0.0,0.0
H,3,LL,0,1,0,0,0,0,0,1.0,0.00,0.0,0.0,0,0.0,0.0
OH,4,LL,1,1,0,0,0,0,0,1.0,0.00,0.0,0.0,0,0.0,0.0


In [122]:
def modify_concentrations(df, concentrations):
    for species, conc in concentrations.items():
        df.at[species, 'LBOUND'] = 2
        df.at[species, 'FIXEDMR'] = conc
    return df

In [125]:
species_df = modify_concentrations(longlived_df, {'O2' : 0.23})

In [126]:
species_df.head()

,order,LONG-LIVED,O,H,C,S,N,CL,LBOUND,VDEP0,FIXEDMR,SGFLUX,DISTH,MBOUND,SMFLUX,VEFF0
species,,,,,,,,,,,,,,,,
O,0,LL,1,0,0,0,0,0,0,1.0,0.00,0.0,0.0,0,0.0,0.0
O2,1,LL,2,0,0,0,0,0,2,0.0,0.23,0.0,0.0,0,0.0,0.0
H2O,2,LL,1,2,0,0,0,0,0,0.0,0.00,0.0,0.0,0,0.0,0.0
H,3,LL,0,1,0,0,0,0,0,1.0,0.00,0.0,0.0,0,0.0,0.0
OH,4,LL,1,1,0,0,0,0,0,1.0,0.00,0.0,0.0,0,0.0,0.0


In [129]:
def modify_flux(df, fluxes):
    for species, flux in fluxes.items():
        df.at[species, 'LBOUND'] = 3
        df.at[species, 'SGFLUX'] = flux
    return df

In [130]:
species_df = modify_flux(longlived_df, {'O2' : 5.0})

In [139]:

other_df

,order,LONG-LIVED,O,H,C,S,N,CL,FIXEDMR
species,,,,,,,,,
HNO2,61,SL,2,1,0,0,1,0,NaN
O1D,62,SL,1,0,0,0,0,0,NaN
CH21,63,SL,0,2,1,0,0,0,NaN
CH23,64,SL,0,2,1,0,0,0,NaN
C2H5,65,SL,0,5,2,0,0,0,NaN
SO21,66,SL,2,0,0,1,0,0,NaN
SO23,67,SL,2,0,0,1,0,0,NaN
HSO3,68,SL,3,1,0,1,0,0,NaN
OCS2,69,SL,1,0,1,2,0,0,NaN


In [138]:
for index, row in df.iterrows():
    print(index)
    print(index, row)

0
0 order          0
species        O
LONG-LIVED    LL
O              1
H              0
C              0
S              0
N              0
CL             0
LBOUND         0
VDEP0          1
FIXEDMR        0
SGFLUX         0
DISTH          0
MBOUND         0
SMFLUX         0
VEFF0          0
Name: 0, dtype: object
1
1 order            1
species         O2
LONG-LIVED      LL
O                2
H                0
C                0
S                0
N                0
CL               0
LBOUND           1
VDEP0            0
FIXEDMR       0.27
SGFLUX           0
DISTH            0
MBOUND           0
SMFLUX           0
VEFF0            0
Name: 1, dtype: object
2
2 order           2
species       H2O
LONG-LIVED     LL
O               1
H               2
C               0
S               0
N               0
CL              0
LBOUND          0
VDEP0           0
FIXEDMR         0
SGFLUX          0
DISTH           0
MBOUND          0
SMFLUX          0
VEFF0           0
Name: 2, dtype: object
3

In [136]:
for row in df.itertuples():
    print(row)

Pandas(Index=0, order=0, species='O', _3='LL', O=1, H=0, C=0, S=0, N=0, CL=0, LBOUND=0.0, VDEP0=1.0, FIXEDMR=0.0, SGFLUX=0.0, DISTH=0.0, MBOUND=0.0, SMFLUX=0.0, VEFF0=0.0)
Pandas(Index=1, order=1, species='O2', _3='LL', O=2, H=0, C=0, S=0, N=0, CL=0, LBOUND=1.0, VDEP0=0.0, FIXEDMR=0.27, SGFLUX=0.0, DISTH=0.0, MBOUND=0.0, SMFLUX=0.0, VEFF0=0.0)
Pandas(Index=2, order=2, species='H2O', _3='LL', O=1, H=2, C=0, S=0, N=0, CL=0, LBOUND=0.0, VDEP0=0.0, FIXEDMR=0.0, SGFLUX=0.0, DISTH=0.0, MBOUND=0.0, SMFLUX=0.0, VEFF0=0.0)
Pandas(Index=3, order=3, species='H', _3='LL', O=0, H=1, C=0, S=0, N=0, CL=0, LBOUND=0.0, VDEP0=1.0, FIXEDMR=0.0, SGFLUX=0.0, DISTH=0.0, MBOUND=0.0, SMFLUX=0.0, VEFF0=0.0)
Pandas(Index=4, order=4, species='OH', _3='LL', O=1, H=1, C=0, S=0, N=0, CL=0, LBOUND=0.0, VDEP0=1.0, FIXEDMR=0.0, SGFLUX=0.0, DISTH=0.0, MBOUND=0.0, SMFLUX=0.0, VEFF0=0.0)
Pandas(Index=5, order=5, species='HO2', _3='LL', O=2, H=1, C=0, S=0, N=0, CL=0, LBOUND=0.0, VDEP0=1.0, FIXEDMR=0.0, SGFLUX=0.0, DISTH=0